<a href="https://colab.research.google.com/github/btcnhung1299/hcmus-computational-graph/blob/master/computational_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import functools

def add_method(cls):
  def decorator(func):

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
      return func(*args, **kwargs)
    
    setattr(cls, func.__name__, wrapper)
    return wrapper
  
  return decorator

### Computational graph architecture

Computational graph is a directed acyclic graph which contains of a set of nodes. Thus, for an edge from `u` to `v`, we define `u` as the parent of `v`.

In [2]:
class Graph:
  def __init__(self):
    self.nodes = set()

In [3]:
class Node:
  def __init__(self):
    self.parent = None
    self.value = None
    self.grad_value = {}
  
  def set_parent(self, parent):
    self.parent = parent

Each node can be either operation or variable. In fact, a variable can be regarded as a special operation which acts as a placeholder.

In [4]:
class Operation(Node):
  def __init__(self, inputs=[]):
    super().__init__()
    self.inputs = inputs

  def compute_grad_value(self, arg_node):
    pass

  def compute_grad_params(self, arg_node):
    for node in self.inputs:
      node.compute_grad_value(arg_node)

In [5]:
class Variable(Operation):
  def __init__(self, symbol):
    super().__init__()
    self.symbol = symbol

  def set_value(self, value):
    self.value = value

  def __repr__(self):
    return '{}'.format(self.symbol)

  def compute_value(self):
    if self.value is None:
      raise ValueError('{} does not carry any value.'.format(self))

  def compute_grad_value(self, arg_node):
    self.grad_value[arg_node] = (1 if self == arg_node else 0) 

`Operation` is an abstract class. To support primary operations such as `+, -, *, /` (ternary operations) or `x^n, e^x, sin(x), cos(x), tan(x)` (unary operations), we need to override them.

#### Ternary Operations

1. **Addition**: Adding any two variables/operations results in an `AddOperation`. Given input nodes whose values provided:
*   `compute_value`: We compute value of the `AddOperation` by simply adding values of the nodes.
*   `compute_grad_value`: We compute gradient value of the `AddOperation` with respect to each input nodes by taking derivation of each.
Let $a = f(x, y)$ and $b = f(x, y)$. $c = f(a, b) = a + b$ for addition.
Then, $\frac{\delta c}{\delta x} = \frac{\delta a}{\delta x} + \frac{\delta b}{\delta x}$. The same holds for $\frac{\delta c}{\delta y}$.

2. We do the same thing to **substraction**, or `SubOperation`.

In [6]:
class AddOperation(Operation):
  def __repr__(self):
    return '{} + {}'.format(self.inputs[0], self.inputs[1])

  def compute_value(self):
    assert len(self.inputs) == 2
    self.value = self.inputs[0].value + self.inputs[1].value

  def compute_grad_value(self, arg_node):
    self.compute_grad_params(arg_node)
    self.grad_value[arg_node] = self.inputs[0].grad_value[arg_node] + self.inputs[1].grad_value[arg_node]

@add_method(Node)
def __add__(self, other):
  add_op = AddOperation(inputs=[self, other])
  return add_op

In [7]:
class SubOperation(Operation):
  def __repr__(self):
    return '{} - {}'.format(self.inputs[0], self.inputs[1])

  def compute_value(self):
    assert len(self.inputs) == 2
    self.value = self.inputs[0].value - self.inputs[1].value

  def compute_grad_value(self, arg_node):
    self.compute_grad_params(arg_node)
    self.grad_value[arg_node] = self.inputs[0].grad_value[arg_node] - self.inputs[1].grad_value[arg_node]

@add_method(Node)
def __sub__(self, other):
  sub_op = SubOperation(inputs=[self, other])
  return sub_op

3. **Multiplication**:
*   `compute_grad_value`: 
Let $a = f(x, y)$ and $b = f(x, y)$. $c = f(a, b) = a * b$.
Then, $\frac{\delta c}{\delta x} = b * \frac{\delta a}{\delta x} + a * \frac{\delta b}{\delta x}$. The same holds for $\frac{\delta c}{\delta y}$.




In [8]:
class MulOperation(Operation):
  def __repr__(self):
    return '{} * {}'.format(self.inputs[0], self.inputs[1])

  def compute_value(self):
    assert len(self.inputs) == 2
    self.value = self.inputs[0].value * self.inputs[1].value

  def compute_grad_value(self, arg_node):
    self.compute_grad_params(arg_node)
    self.grad_value[arg_node] = self.inputs[0].grad_value[arg_node] * self.inputs[1].value + \
                                self.inputs[1].grad_value[arg_node] * self.inputs[0].value

@add_method(Node)
def __mul__(self, other):
  mul_op = MulOperation(inputs=[self, other])
  return mul_op

4. **Division**:
*   `compute_grad_value`: 
Let $a = f(x, y)$ and $b = f(x, y)$. $c = f(a, b) = \frac{a}{b}$.
Then, $\frac{\delta c}{\delta x} = \frac{1}{b} * \frac{\delta a}{\delta x} - \frac{a}{b^2} * \frac{\delta b}{\delta x}$. The same holds for $\frac{\delta c}{\delta y}$.

In [9]:
class DivOperation(Operation):
  def __repr__(self):
    return '{} / {}'.format(self.inputs[0], self.inputs[1])

  def compute_value(self):
    assert len(self.inputs) == 2
    self.value = self.inputs[0].value / self.inputs[1].value

  def compute_grad_value(self, arg_node):
    self.compute_grad_params(arg_node)
    self.grad_value[arg_node] = self.inputs[0].grad_value[arg_node] / self.inputs[1].value - \
                                self.inputs[0].value * self.inputs[1].grad_value[arg_node] / (self.inputs[1].value ** 2)

@add_method(Node)
def __truediv__(self, other):
  div_op = DivOperation(inputs=[self, other])
  return div_op

#### Unary operations

In [20]:
class PowOperation(Operation):
  def __init__(self, inputs, const):
    super().__init__(inputs)
    self.const = const

  def __repr__(self):
    return '{}^{}'.format(self.inputs[0], self.const)

  def compute_value(self):
    assert len(self.inputs) == 1
    self.value = self.inputs[0].value ** self.const

  def compute_grad_value(self, arg_node):
    self.compute_grad_params(arg_node)
    self.grad_value[arg_node] = self.const * self.inputs[0].grad_value[arg_node] * (self.inputs[0].value ** (self.const - 1))


@add_method(Node)
def __pow__(self, power):
  pow_op = PowOperation(inputs=[self], const=power)
  return pow_op


### Auto-diff

To sequentially compute value of a given equation, we sort nodes in computational graph by topologically ordering. To obtain gradient value, we need to firstly compute all values in the graph and sequentially compute gradient value of the target node with respect to argument nodes in reverse topological order.

In [21]:
def topo_sort(nodes):

  def dfs(node, visited_nodes, topo_order):
    if node in visited_nodes:
      return
    
    visited_nodes.add(node)
    for node_input in node.inputs:
      if node_input not in visited_nodes:
        dfs(node_input, visited_nodes, topo_order)

    topo_order.append(node)

  topo_order = []
  visited_nodes = set()

  for node in nodes:
    dfs(node, visited_nodes, topo_order)
  
  return topo_order

In [22]:
def compute_values(target_nodes):
  topo_order = topo_sort(target_nodes)
  for node in topo_order:
    node.compute_value()
  return {node: node.value for node in target_nodes}

In [13]:
def compute_grad_values(target_node, arg_nodes):
  rev_topo_order = topo_sort([target_node])
  rev_topo_order.reverse()
  
  for node in rev_topo_order:
    target_node.compute_grad_value(node)
  return {node: target_node.grad_value[node] for node in arg_nodes}

### Test

In [23]:
x = Variable(symbol = 'x')
b = x ** 2
x.set_value(4)
compute_grad_values(b, [x])

{x: 8}

In [15]:
x = Variable(symbol = 'x')
y = Variable(symbol = 'y')
a = x + y
b = x
c = a / b

x.set_value(2)
y.set_value(3)

In [16]:
compute_values([c])
compute_grad_values(c, [x, y])

{x: -0.75, y: 0.5}